In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import seaborn as sns

In [2]:
df =  pd.read_csv('titanic-2.csv')

In [3]:
titanic_data = df.drop(['Name','Ticket','Cabin'],axis=1)

In [4]:
titanic_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,NaN,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [5]:
titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

C:\Users\jay\AppData\Local\Temp\ipykernel_8952\4154180430.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)
C:\Users\jay\AppData\Local\Temp\ipykernel_8952\4154180430.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [6]:
label_encoder = LabelEncoder()
titanic_data['Sex'] = label_encoder.fit_transform(titanic_data['Sex'])
titanic_data['Embarked'] = label_encoder.fit_transform(titanic_data['Embarked'])


In [7]:
X = titanic_data.drop('Survived',axis=1)
y = titanic_data['Survived']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
clf = DecisionTreeClassifier(random_state=42)

In [10]:
param_grid = {
    'criterion': ['gini','entropy','log_loss'],
    'max_depth' : [None,10,15,20,30],
    'min_samples_split' : [7,8,10],
    'min_samples_leaf' : [20,22,25,25],
}

In [11]:
gridSearch = GridSearchCV(estimator=clf,param_grid=param_grid,cv=5,scoring='accuracy',n_jobs=1)
gridSearch.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 10, 15, 20, 30],
                         'min_samples_leaf': [20, 22, 25, 25],
                         'min_samples_split': [7, 8, 10]},
             scoring='accuracy')

In [12]:
best_parms = gridSearch.best_params_
print(best_parms)

{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 22, 'min_samples_split': 7}


In [13]:
best_clf = DecisionTreeClassifier(random_state=42,**best_parms)
best_clf.fit(X_train,y_train)

DecisionTreeClassifier(min_samples_leaf=22, min_samples_split=7,
                       random_state=42)

In [14]:
y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.7932960893854749